# Assignment 2.2 - Vehicle Routing Problem
JM0100-M-6 Business Analytics  
Myrthe Wouters  
u1273195

In [1]:
# Global imports
import pandas as pd
import numpy as np
from haversine import haversine, Unit
import math
import operator
from copy import deepcopy

In [2]:
# Load data
STORES = pd.read_excel('Data Excercise 2 - EMTE stores - BA 2020-1.xlsx', index_col='City Nr.')
# Add column that defines if location is Jumbo
STORES['is_jumbo'] = STORES.apply(lambda row: row['Type']=='Jumbo', axis=1)

In [3]:
STORES.head()

,Name,Address,Postal code,City,Lat,Long,Type,is_jumbo
City Nr.,,,,,,,,
0,EMTE HEADQUARTERS VEGHEL,CORRIDOR 11,5466RB,VEGHEL,51.606702,5.528046,NaN,False
1,EMTE ARKEL,DR H DE VRIESPLN 14,4241BW,ARKEL,51.864000,4.993040,Coop,False
2,EMTE ARNEMUIDEN FR,CLASINASTR 5,4341ER,ARNEMUIDEN,51.500010,3.677280,Jumbo,True
3,EMTE BATHMEN FR,LARENSEWG 18,7437BM,BATHMEN,52.249060,6.289990,Jumbo,True
4,EMTE BEEK EN DONK,HEUVELPLN 73,5741JJ,BEEK EN DONK,51.529300,5.632300,Jumbo,True


## Exercise 2.2
Write a local search improvement heuristic to improve the solution you have created in Exercise 2.1.

### Definitions
* I consider a __route__ as one day for John.
* I consider a __schedule__ as the entire plan in which every store is visited exactly once, consisting of multiple routes. 

### Calculate rounded distances between every two locations
I will use a matrix to store the distances and travel times between every two stores. This makes my algorithms more efficient, because I do not have to call the haversine formula every time I want to compute the distance between two stores.

In [4]:
def calc_dist(loc1, loc2):
    '''Defines rounded distance (km) between two locations'''
    coords1 = STORES.loc[loc1, 'Lat'], STORES.loc[loc1, 'Long']
    coords2 = STORES.loc[loc2, 'Lat'], STORES.loc[loc2, 'Long']
    dist = round(haversine(coords1, coords2))
    return dist

In [5]:
def dist_to_min(km, speed_kmh):
    """Defines the duration of a route in minutes"""
    speed_kmm = speed_kmh/60
    minutes = round(km/speed_kmm)
    return minutes

In [6]:
# Save all distance in global variable DIST_MATRIX and all travel_times in global variable TIME_MATRIX
DIST_MATRIX = np.zeros((len(STORES), len(STORES)))
TIME_MATRIX = np.zeros((len(STORES), len(STORES)))

for location_1 in STORES.index:
    for location_2 in STORES.index:
        dist = calc_dist(location_1, location_2)
        DIST_MATRIX[location_1, location_2] = dist
        TIME_MATRIX[location_1, location_2] = dist_to_min(dist, 90)

For exercise 2.2, we have to do a lot of swap and move operations. Therefore, for efficiency reasons, I defined another Route class - RouteDLL - that stores a route as a doubly linked list.

### Classes

#### Location Class

In [7]:
class Location:
    
    def __init__(self, nr, name, lat, long, store_type):
        self.nr = nr
        self.name = name
        self.lat = lat
        self.long = long
        self.store_type = store_type
        self.visited = False
        self.distance_hq = DIST_MATRIX[0, nr]
    
    @property
    def is_jumbo(self):
        '''Boolean value that shows if location is a Jumbo location'''
        return True if self.store_type=='Jumbo' else False

In [8]:
def get_params(df, index):
    '''
    Function to get all necessary variables from STORES dataframe to instantiate a Location instance for every store
    '''
    params = df.iloc[index][['Name', 'Lat', 'Long', 'Type']].values
    name, lat, long, store_type = params
    return index, name, lat, long, store_type

In [9]:
# Define global variable HQ
HQ = Location(*get_params(STORES, 0))

#### Node Class
As mentioned before, I decided to implement routes in exercise 2.2 as doubly linked list. For a doubly linked list, we need a Node class that stores data of a node along with previous and next pointers. The data of each node is an instance of Location class.

In [10]:
class Node:
    def __init__(self, data):
        self.data = data
        self.prev = None
        self.next = None

#### RouteDLL Class

As mentioned before, I define a RouteDLL class for routes with their individual attributes and methods. 

*There are a few important attributes, properties and methods of the RouteDLL class to highlight:*

* the `self.start` and `self.end` attributes of a DLL route represent the start and end and the HQ respectively
* the `inner_distance_time` method calculates the distance and travel time of the inner route, where inner route has the same definition as in Exercise 2.1
* the `total_distance_time` method calculates the distance and travel time of the full route, where full route has the same definition as in Exercise 2.1
* the `is_valid` property defines if the route is valid, i.e., meets all of the following three constraints:
    * `store_type_constraint`: all stores in the route should be of the same type
    * `working_hours_constraints`: the entire duration of the route should be less than 11 hours
    * `opening_hours_constraints`: all stores in the route should be visited between 09:00-17:00
* the `insert_after` method inserts a node after a given node in a route
* the `remove` method removes a node from a route

In [11]:
class RouteDLL:
    
    # Every route starts and ends at HQ
    hq_start = Node(HQ)
    hq_end = Node(HQ)
    
    # Class variables
    max_working_mins = 11*60
    max_opening_mins = 8*60
    
    def __init__(self, is_jumbo):
        self.start = RouteDLL.hq_start
        self.end = RouteDLL.hq_end
        self.start.next = self.end
        self.end.prev = self.start
        self.is_jumbo = is_jumbo
        
    @property
    def meeting_time(self):
        '''Defines the meeting time at individiual stores in the route'''
        return 30 if self.is_jumbo else 20
    
    @property
    def length(self):
        '''Defines number of stores (including HQ) in the route'''
 
        temp = self.start # Initialise temp 
        count = 0 # Initialise count 
  
        # Loop while end of linked list is not reached 
        while (temp): 
            count += 1
            temp = temp.next
            
        return count 
    
    @property
    def full_route(self):
        '''Returns the order of store numbers in the full route as a list'''
        
        temp = self.start
        route = []
        
        while (temp):
            route += [temp.data.nr]
            temp = temp.next
            
        return route
    
    @property
    def total_visit_time(self):
        '''Defines total time spend in all stores of route together'''
        return ((self.length)-2) * self.meeting_time # Subtract 2 from self.length, as we do not have a meeting time at the HQ
    
    @property
    def store_type_constraint(self):
        '''
        Defines if all stores in route are of same type.
        More precisely, defines if all stores in a route are either all of type Jumbo or all of type Coop/other. 
        Returns True if a route meets this store type constraint, else False.
        '''
        temp = self.start.next
        
        while temp != self.end:
            if temp.data.is_jumbo != self.is_jumbo:
                return False
            temp=temp.next

        return True
    
    @property
    def working_hours_constraint(self):
        '''
        Defines if route meets the constraint of John's working hours. John cannot work more than 11 hours = 11*60 minutes per day.
        Hence, this method checks if the total travel time of the route and the total visit time at the stores does not exceed 11 hours.
        '''
        return (self.total_distance_time()['total_travel_time'] + self.total_visit_time) <= RouteDLL.max_working_mins
    
    @property
    def opening_hours_constraint(self):
        '''
        Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route.
        This means that traveling time from the first store to the last store in the route (i.e., the inner route) plus total visiting time
        at the stores does not exceed 8 hours.
        '''
        return (self.inner_distance_time()['total_travel_time'] + self.total_visit_time) <= RouteDLL.max_opening_mins
    
    @property
    def is_valid(self):
        '''
        Defines if route meets all constraints, that is:
        * Constraint of John's working hours (11 hours max)
        * Constraint of visiting hours at every store (9:00-17:00, that is, 8 hours max)
        * Constraint of every store in route being either of type Jumbo or of type Coop/other
        '''
        return self.working_hours_constraint and self.opening_hours_constraint and self.store_type_constraint
    
    @property
    def total_distance(self):
        '''Defines total distance of total route'''
        return self.total_distance_time()['total_distance']
    
    @property
    def cumsum(self):
        '''
        Defines cumulative sum of distances at each location.
        We need this cumulative sum in order to convert the final schedule to an Excel file.
        '''
        temp = self.start # Initialise temp
        distance = 0
        cumsum = []
        
        while temp.next:
            # Calculate distance between current and next node
            dist = DIST_MATRIX[temp.data.nr, temp.next.data.nr]
            
            # Add dist to distance
            distance += dist
            #Append cumulative distance until this node to cumsum list
            cumsum.append(distance)
            
            # Move to next node
            temp = temp.next
        
        # Insert cumsum of 0 at start at HQ
        cumsum.insert(0, 0)
        return cumsum
    
    def total_distance_time(self):
        '''Defines total distance and travel time of the full route (i.e., route including start and end at HQ)'''
        temp = self.start # Initialise temp
        distance = 0
        travel_time = 0
        
        while temp.next:
            # Calculate distance and time between current and next node
            dist = DIST_MATRIX[temp.data.nr, temp.next.data.nr]
            time = TIME_MATRIX[temp.data.nr, temp.next.data.nr]
            
            # Add dist and time to distance and travel_time
            distance += dist
            travel_time += time
            
            # Move to next node
            temp = temp.next
            
        return {'total_distance': distance,
                'total_travel_time': travel_time}
    
    def inner_distance_time(self):
        '''Defines total distance and travel time of inner route (i.e., route excluding start and end at HQ)'''
        temp = self.start.next # Initialise temp
        distance = 0
        travel_time = 0
        
        while temp.next != self.end:
            # Calculate distance and time between current and next node
            dist = DIST_MATRIX[temp.data.nr, temp.next.data.nr]
            time = TIME_MATRIX[temp.data.nr, temp.next.data.nr]
            
            # Add dist and time to distance and travel_time
            distance += dist
            travel_time += time
            
            # Move to next node
            temp = temp.next
        
        return {'total_distance': distance,
                'total_travel_time': travel_time}    
        
    def insert_after(self, prev_node, new_node):
        '''Insert node at given position, i.e., after anothet node, in route'''
        
        # Check if prev_node exists
        if prev_node is None:
            print("This node doesn't exist in DLL")
            return
        
        # Make next of new node as next of prev_node
        new_node.next = prev_node.next
        
        # Make the next node of prev_node as new_node
        prev_node.next = new_node
        
        # Make the prev_node as previous of new_node
        new_node.prev = prev_node
        
        # Change previous of new_node's next node
        if new_node.next is not None:
            new_node.next.prev = new_node
            
    def remove(self, dele):
        '''Remove node from route'''
        
        # Change previous pointer of dele's next node:
        dele.next.prev = dele.prev 
        
        # Change next pointer of dele's previous node:
        dele.prev.next = dele.next
        
        # Set dele's previous and next pointers to None
        dele.next=None
        dele.prev=None

#### Solver Class 

The Solver Class includes attributes and methods to implement the local search improvement heuristic.

*The local search improvement heuristic through the Solver class is implemented in the following way:*
1. Define routes $r$ that changed since last iteration (initialise $r$ with all routes)
2. Compute all possible swaps and moves for each node in each route $r$ and save all swaps and moves that lead to an improvement and are valid
3. As long as there are valid swaps/move that lead to an improvement, compute the swap or move that leads to the highest improvement. If there are no swaps/moves that lead to an improvement anymore, break the loop
4. Update routes $r$ to routes that changed in this iteration
5. Delete saved improvements from nodes in routes in $r$ 
6. Continue at step 1

*There are a few important attributes, properties and methods of the Solver class to highlight:*
* the `planned_stores` attribute represents current schedule, initialised with result of exercise 2.1
* the `all_routes_valid` property checks if all routes in the schedule are valid
* the `total_distance` property defines the total distance of the schedule
* the `all_swaps` and `all_moves` methods compute all swaps and moves respectively. Take an argument `changed_routes`. Only for nodes in these routes, all swaps/moves are computed
* the `optimize` method computes the entire local search improvement heuristic as described above

In [12]:
class Solver:
    
    def __init__(self, planned_routes):
        self.planned_routes = planned_routes
        
    @property
    def all_routes_valid(self):
        '''Checks if all planned routes in the schedule are valid, i.e., meets all constraints'''
        for route in self.planned_routes:
            if not route.is_valid:
                return False
        return True
    
    @property
    def total_distance(self):
        '''Defines total distance of all planned routes together'''
        dist = 0
        for route in self.planned_routes:
            dist += route.total_distance
        return dist
        
    @staticmethod
    def swap_nodes(node_1, node_2):
        '''Swaps node 1 with node 2, i.e., swap data of node 1 with data of node 2'''
        # Nothing to do if x and y are same 
        if node_1 == node_2: 
            return 
    
        temp = deepcopy(node_1.data)
        
        node_1.data = node_2.data
        node_2.data = temp
        
    @staticmethod
    def move_node_after(node, node_to_move_after):
        '''Move position of node to position after node_to_move_after'''
        
        # If node is is equal to node_to_move_after, do nothing
        if node == node_to_move_after:
            return
        
        # Remove node at original position
        ## Change node's previous node next pointer to node's next node
        node.prev.next = node.next
        
        ## Change node's next node previous pointer to node's previous node
        node.next.prev = node.prev
        
        # Insert node at new position
        ## Make next of new node as next of prev_node
        node.next = node_to_move_after.next
        
        ## Make the next node of prev_node as new_node
        node_to_move_after.next = node
        
        ## Make the prev_node as previous of new_node
        node.prev = node_to_move_after
        
        ## Change previous of new_node's next node
        node.next.prev = node
        
    @staticmethod
    def undo_move(route_org, route_curr, node, orig_before_node):
        '''Undo the move of node'''
        
        route_curr.remove(node)
        route_org.insert_after(orig_before_node, node)
    
    def all_swaps(self, changed_routes):
        '''
        Computes all possible swaps for all nodes in planned routes.
        This method only does forward swaps, as swapping node 1 with node 2 implies the same as swapping node 2 with node 1.
        Takes an argument changed_routes. The method only computes all possible swaps for nodes within changed_routes.
        '''
        
        # Dictionary to store the swaps that lead to new improvements in this iteration
        improvements = {}
        
        curr_dist = deepcopy(self.total_distance) # store the distance of all planned routes now together 
        
        # Check for each location
        # I use enumerate here, to check which route we are currently in, so that we can implement forward swaps only
        for idx_route, route in enumerate(self.planned_routes):
            # Set node to compute all swaps with
            curr_loc = route.start.next
            
            while curr_loc != route.end:
                # Compute only swaps with nodes in current route and routes after, in order to prevent doing double work
                for check_route in self.planned_routes[idx_route:]:
                    
                    # If none of the routes the two nodes are in changed compared to the last iteration,
                    # we have saved the improvement of the swap and thus do not have to compute it again
                    if (route not in changed_routes) and (check_route not in changed_routes):
                        continue
                    
                    # If any of the routes the two nodes are in changed, we have to recompute the improvement of the 
                    # swap
                    else: 
                        if route == check_route:
                            check_loc = curr_loc.next # Only check for nodes after current node
                        else:
                            check_loc = check_route.start.next
                        
                        while check_loc != check_route.end:
                            Solver.swap_nodes(curr_loc, check_loc) # swap two nodes
                        
                            improv = curr_dist - (self.total_distance) # check if there is an improvement
                            
                            # If the swap leads to an improvement and results in all valid routes 
                            # save improvement and details in improvements dictionary
                            if (improv > 0) and self.all_routes_valid:
                                improvements[(curr_loc, check_loc)] = {}
                                improvements[(curr_loc, check_loc)]['improv'] = improv
                                improvements[(curr_loc, check_loc)]['routes'] = (route, check_route)
                            
                            # Swap nodes back
                            Solver.swap_nodes(check_loc, curr_loc)
                            
                            # Go on with next location to swap with
                            check_loc = check_loc.next
                            
                # Check all swaps for next location
                curr_loc = curr_loc.next
        return improvements
    
    def all_moves(self, changed_routes):
        '''
        Computes all possible moves for all nodes in planned routes.
        Takes an argument changed_routes. The method only computes all possible moves for nodes within changed_routes.
        '''
        
        # Dictionary to store the moves that lead to new improvements in this iteration
        improvements = {}
        
        curr_dist = deepcopy(self.total_distance) # store the distance of all planned routes now together 
        
        # Check for each location
        for route in self.planned_routes:
            # Set node to compute all moves for
            curr_loc = route.start.next
            
            while curr_loc != route.end:
                # Save the node before the current node we are looking at, in order to be able to undo the move
                orig_before_node = curr_loc.prev
                
                for check_route in self.planned_routes:
                    # Set node we will move curr_loc after
                    move_after = check_route.start
                    
                    # If none of the routes the two nodes are in changed compared to the last iteration,
                    # we have saved the improvement of the move and thus do not have to compute it again
                    if (route not in changed_routes) and (check_route not in changed_routes):
                        continue
                    
                    else:
                        while move_after != check_route.end:
                        
                            Solver.move_node_after(curr_loc, move_after) # move curr_loc after move_after

                            improv = curr_dist - (self.total_distance) # check if there is an improvement
                            
                            # If the move leads to an improvement and results in all valid routes,
                            # save improvement and details in improvements dictionary
                            if (improv > 0) and self.all_routes_valid:
                                improvements[(curr_loc, move_after)] = {}
                                improvements[(curr_loc, move_after)]['improv'] = improv
                                improvements[(curr_loc, move_after)]['routes'] = (route, check_route)
                            
                            # Undo the move
                            solver.undo_move(route, check_route, curr_loc, orig_before_node)
                            
                            # Move to next location to move after
                            move_after = move_after.next
                
                # Check all moves for next location
                curr_loc = curr_loc.next

        return improvements
    
    def optimize(self):
        '''
        This method executes the local search improvement heuristic.
        In order to make the code more efficient, some clever logic is iimplemented such that we do not have to compute all 
        possible move and swap options again in each round, but update only those for which the corresponding routes are changed in the 
        previous round.
        This logic is implemented by storing the swap and move improvements respectively in the swap_improv and move_improv dictionaries. 
        As soon as the best move or swap is implemented, the changed routes through that move or swap are stored in the variable changed_routes.
        Then, swaps and moves with nodes that are in changed_routes are deleted from the the swap_improv and move_improv dictionaries. In the next
        iteration, the move and swap improvements dictionaries are updated by computing all swaps and moves for the routes in changed_routes. 
        For the swaps and moves within routes that have not changed, the swap_improv and move_improv still store all swaps and moves that lead to 
        an improvement and therefore do not require new computations.
        '''
        
        operations = 0 # Initialise nr of operation to lead to optimal result
        swap_improv = {} # Dictionary to save all swap improvements
        move_improv = {} # Dictionary to save all move improvements
        
        # Initialise routes that changed since last iteration. For the first iteration, we have not computed any moves or swaps yet.
        # Therefore, we have to compute all possible moves and swaps for each route.
        changed_routes = self.planned_routes 
        
        while True:
            # Compute all move and swaps for this iteration
            iter_swap_improv = self.all_swaps(changed_routes)
            iter_move_improv = self.all_moves(changed_routes)
            
            # Update dictionaries with swap and move improvements with improvements of changed routes 
            # during last iteration
            swap_improv.update(iter_swap_improv)
            move_improv.update(iter_move_improv)
            
            # Sort swap improvements on improvement value decreasing
            swap_improv_list = [(values['improv'], swap, values['routes']) for swap, values in swap_improv.items()]
            swap_improv_sorted = sorted(swap_improv_list, key=operator.itemgetter(0), reverse = True)
            
            # If there is a swap that leads to an improvement, set best_swap
            if len(swap_improv_sorted) > 0:
                # The best swap improvement is the first swap tuple in the swap_improv_sorted list. 
                best_swap = swap_improv_sorted[0] 
            else:
                best_swap = None # Else set to None, as there is no swap improvement
            
            # Sort move improvements on improvement value decreasing
            move_improv_list = [(values['improv'], move, values['routes']) for move, values in move_improv.items()]
            move_improv_sorted = sorted(move_improv_list, key=operator.itemgetter(0), reverse = True)
            
            # If there is a move that leads to an improvement, set best_move 
            if len(move_improv_sorted) > 0:
                # The best move improvement is the first move tuple in the move_improv_sorted list. 
                best_move = move_improv_sorted[0]
            else:
                best_move = None # Else set to None, as there is no move improvement
            
            # If there is no swap or move improvement anymore, we have found the optimal solution of the local search improvement heuristic
            # Hence, we can break the loop.
            if (len(swap_improv) + len(move_improv))==0:
                break
            
            # If the best improvement is a move improvement, compute the best move
            if best_move:
                if (not best_swap) or (best_move[0] > best_swap[0]):
                    # The second element in the best move tuple stores another tuple with the node to move and the node to move it after.
                    self.move_node_after(*best_move[1])
                    operations+=1 # Increase number of operations by 1
                    print('Moved {} after {}'.format(best_move[1][0].data.name, best_move[1][1].data.name))
                
                    # Keep track of the routes that have changed during the move operation
                    changed_routes = list(best_move[2])
                
            
            # If the best improvement is a swap improvement, compute the best swap
            if best_swap:
                if (not best_move) or (best_swap[0] >= best_move[0]):
                    # The second element in the best swap tuple stores another tuple with the two nodes to swap.
                    self.swap_nodes(*best_swap[1])
                    operations+=1 # increase number of operations by 1
                    print('Swapped {} with {}'.format(best_swap[1][0].data.name, best_swap[1][1].data.name))
                
                    # Keep track of the routes that have changed during the swap operation
                    changed_routes = list(best_swap[2])
                
            # Delete moves in changed_routes from move_improv dictionary, as they have to be computed again in 
            # next iteration
            for move in move_improv_sorted:
                # For each move tuple in the move_improv_sorted list, the routes that the two nodes belong to are stored in 
                # the third element of this tuple. Hence, for each move improvement, check if any of the routes the nodes were in 
                # have changed. If so, delete from move_improv dictionary as they need to be updated in next iteration.
                if any(True for route in changed_routes if route in move[2]):
                    del move_improv[move[1]]
                
            # Delete swaps in changed_routes from swap_improv dictionary, as they have to be computed again in 
            # next iteration
            for swap in swap_improv_sorted:
                # For each swap tuple in the swap_improv_sorted list, the routes that the two nodes belong to are stored in 
                # the third element of this tuple. Hence, for each swap improvement, check if any of the routes the nodes were in 
                # have changed. If so, delete from swap_improv dictionary as they need to be updated in next iteration.
                if any(True for route in changed_routes if route in swap[2]):
                    del swap_improv[swap[1]]
                        
        print('\nOptimized distance after local search improvement heuristic = {} in {} move/swap operations'.format(
            solver.total_distance, operations))
        
        # Make dictionary with route number and key and RouteDLL instance as value
        routes = {}
        i = 1
        for route in self.planned_routes:
            routes[i] = route
            i += 1
        
        return routes

### Import initial solution of 2.1 and set to RouteDLL instances
In order to execute the local search improvement heuristic in exercise 2.2, I have to set the routes in the initial solution of exercise 2.1 to RouteDLL instances.

In [13]:
# Read from excel file exercise 1
solution_1 = pd.read_excel('Ex2.1-1273195.xls')
solution_1 = list(solution_1['City Nr.'])[1:-1] # Take store numbers, exact for the first and last HQ

initial_schedule = [] 
route = []
for location in solution_1:
    # Append all store numbers except for headquarters
    if location!=0:
        route.append(location)
    # Once we come accross a headquarter
    else:
        # If the HQ is the end of the route, append this route to initial_schedule
        if len(route)>0:
            initial_schedule.append(route[:])
            route = []
# Append last route to initial_schedule
initial_schedule.append(route[:])
            
initial_solution = []        
for route in initial_schedule:
    # Check if route is a Jumbo or Coop/Other route
    is_jumbo = STORES.loc[route[0], 'is_jumbo']
    
    # Instantiate DLL instance
    route_dll = RouteDLL(is_jumbo)
    curr = route_dll.start # Initialise for insertion
    
    for location in route:
        # Create location instance for location
        loc = Location(*get_params(STORES, location))
        # Create node instance for locaton
        node_loc = Node(loc)
        # Insert node to RouteDLL instance
        route_dll.insert_after(curr, node_loc)
        curr = node_loc # set curr to this node
        
    # Append a deep copy of the RouteDLL instance to a list of planned routes of type RouteDLL
    initial_solution.append(deepcopy(route_dll))

# Check if the transition worked correctly, i.e., check of the total distance of the schedule with RouteDLL instances equals the total
# distance of the initial solution of 2.1 (3019)
sum([route.total_distance for route in initial_solution]) == 3019

True

### Define function to save a schedule to an Excel file

In [14]:
def to_Excel(planned_routes, save=True, file_name=None):
    '''Creates dataframe with needed information from solved VRP, saves to Excel file if needed'''
    
    # Dictionary to store data from all routes
    data = {'Route Nr.': [], 
            'City Nr.': [],
            'City Name': [], 
            'Total Distance in Route (km)': [],
            'Total Distance (km)': []}
    
    # Define total kilometers traveled at start of the route in previous routes
    total_km_at_start = 0
    
    # All routes in planned routes
    for nr, route in planned_routes.items():
        cumsum_loc = 0  # Define current cumulative sum location
  
        temp = route.start # Initialise temp
            
        while (temp):
            data['Route Nr.'].append(nr)
            data['City Nr.'].append(temp.data.nr)
            data['City Name'].append(temp.data.name)
            data['Total Distance in Route (km)'].append(route.cumsum[cumsum_loc])
            data['Total Distance (km)'].append(route.cumsum[cumsum_loc] + total_km_at_start)
                
            temp = temp.next
            
            cumsum_loc += 1 # Update current cumulative sum location
        
        # Update total kilometers traveled at start of the route in previous routes
        total_km_at_start += route.cumsum[-1] 
    
    df = pd.DataFrame.from_dict(data) # Save data to DataFrame
    
    # If needed, save data to Excel file
    if save:
        df.to_excel(file_name, index=False)
            
    return df

### Solution 2.2

In [15]:
# Instantiate solver instance for local search improvement heuristic with solution of exercise 2.1
solver = Solver(initial_solution)
planned_routes_2 = solver.optimize() # Solve local search improvement heuristic

# Save results to Excel file
df = to_Excel(planned_routes_2, save=False)

Moved EMTE ST ANTHONIS after EMTE Montfort FR
Swapped EMTE GROENLO with EMTE RIJSSEN VEENESLAGEN
Swapped EMTE RIJSSEN VEENESLAGEN with EMTE BATHMEN FR
Swapped EMTE UDENHOUT with EMTE KAATSHEUVEL
Moved EMTE Eindhoven victoriapark after EMTE VESSEM FR
Moved EMTE GROENLO after EMTE HEADQUARTERS VEGHEL
Moved EMTE KRUININGEN FR after EMTE HOEK FR
Moved EMTE S GRAVENPOLDER FR after EMTE HOEK FR
Moved EMTE LOON OP ZAND after EMTE HEADQUARTERS VEGHEL
Moved EMTE GROOT AMMERS after EMTE HOORNAAR FR
Moved EMTE TILBURG BESTERDRING after EMTE GILZE
Swapped EMTE ENTER FR with EMTE RIJSSEN OPSTALLSTR
Moved EMTE Dongen after EMTE UDENHOUT
Moved EMTE TILBURG WAGNERPLEIN after EMTE UDENHOUT
Swapped EMTE OSSENDRECHT FR with EMTE PUTTE (NB)
Moved EMTE SCHAIJK after EMTE WIJCHEN
Swapped EMTE VLISSINGEN FR with EMTE KOUDEKERKE FR
Swapped EMTE CUIJK with EMTE LOBITH
Swapped EMTE ST MICHIELSGESTEL with EMTE DEN BOSCH
Moved EMTE OUDENBOSCH after EMTE KRUININGEN FR
Moved EMTE OOLTGENSPLAAT FR after EMTE KRUININ